# MTH3302 - Méthodes probabilistes et statistiques pour I.A.
#### Polytechnique Montréal
### Projet A2024
----
Équipe T - TODO
### Objectif
Prédiction de **la consommation en carburant de voitures récentes**.

### Données
Le jeu de données contient pour presque 400 véhicule, la consommation moyenne en L/100km, l'année de frabrication, le type de véhicule, le nombre de cylindre, cylindree, la transmission et la boite.

- `train.csv` est l'ensemble d'entraînement
- `test.csv` est l'ensemble de test


## Chargement des données

Importation des librairies utilisées dans le calepin.

// TODO: à enlever à la fin:

Pour importer librairies:
using Pkg
Pkg.add(["CSV", "DataFrames", "Combinatorics", "Gadfly", "Distributions"], ...)

In [1]:
# import Pkg; Pkg.add("Plots")
import Pkg; Pkg.add("DecisionTree")

   Resolving package versions...
  No Changes to `C:\Users\Antoine\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Antoine\.julia\environments\v1.10\Manifest.toml`


In [2]:
using CSV, DataFrames, Statistics, Dates, Gadfly, LinearAlgebra, DecisionTree, Random, Plots


In [3]:
function add_rows(data)
    data[!,:volume_gaz] = data[!,:nombre_cylindres] .* data[!,:cylindree]
    return data
end

add_rows (generic function with 1 method)

In [4]:
function getStandardEncodedData(data)
    data_copy = deepcopy(data)
    standardised_data = add_rows(data_copy)
    return standardised_data
end




getStandardEncodedData (generic function with 1 method)

In [5]:
function decisionTree_remise(X_train ,y_train,  X_test, max_depth)
    model = DecisionTreeRegressor(max_depth=max_depth)
    fit!(model, X_train, y_train) #peut etre normaliser sur tout les données, pas justr sur les données d'entrainement
    ychap =  predict(model, X_test)  
    return ychap
end

decisionTree_remise (generic function with 1 method)

In [6]:
function format_file(predictions)
    println(predictions)
    current_time = Dates.format(now(), "yyyy-mm-dd_HH-MM-SS")
    file_name = "benchmark_" * current_time * ".csv"
    CSV.write("./soumissions_potentielles/" *file_name, predictions)
end

format_file (generic function with 1 method)

In [ ]:
COLUMNS_TO_COMPARE = ["annee","type", "nombre_cylindres", "cylindree", "transmission", "boite"]

function get_unique_data(data)
    unique_data = combine(groupby(data, COLUMNS_TO_COMPARE), :consommation => mean)
    rename!(unique_data, :consommation_mean => :consommation)
    return unique_data
end

function find_pairs(data_train, data_predict)
    train_data_without_consommation = select(data_train, Not(:consommation))
    predict_with_annee = data_predict

    common_values = innerjoin(data_train, predict_with_annee, on=names(train_data_without_consommation))

    return common_values
end

function find_differents(data_train, data_predict)
    different_values = antijoin(data_predict, data_train, on=COLUMNS_TO_COMPARE)
    return different_values
    
end

function predict_with_dups(data_train, data_predict)
    pairs = find_pairs(data_train, data_predict)
    pairs = select(pairs, [:id, :consommation])
    different_values = find_differents(data_train, data_predict)

    trainDataStandard = getStandardEncodedData(data_train)
    

    # X_train =  Matrix(trainData[:, Not(:consommation)])
    # y_train = trainData.consommation

    X_test = Matrix(testData)

    X_train = Matrix(trainDataStandard[:, Not(:consommation)])
    y_train = trainDataStandard.consommation
    # println(X_train)
    # println(y_train)

    predictDataStandard = getStandardEncodedData(different_values)

    # X_test = Matrix(predictDataStandard)

    res =  decisionTree_remise(X_train ,y_train,  X_test, 5)

    # different_values = select(different_values, [:id, :consommation])
    # prediction = vcat(pairs, different_values)
    # prediction = sort(prediction, :id)

    # return prediction
end
    

LoadError: UndefVarError: `annee` not defined

In [8]:
trainData = CSV.read("../../data/train.csv", DataFrame)
testData = CSV.read("../../data/test.csv", DataFrame)
trainData.consommation = parse.(Float64,replace.(trainData.consommation, "," => "."))
trainData.cylindree = parse.(Float64,replace.(trainData.cylindree, "," => "."))
testData.cylindree = parse.(Float64,replace.(testData.cylindree, "," => "."))


COMSOMMATION_MEAN = mean(trainData.consommation)
COMSOMMATION_STD = std(trainData.consommation)
trainData[!, :id] = 1:nrow(trainData)
testData[!, :id] = 1:nrow(testData)




1:150

In [9]:
uniqueD = get_unique_data(trainData)

Row,type,nombre_cylindres,cylindree,transmission,boite,consommation
,String31,Int64,Float64,String15,String15,Float64
1,voiture_moyenne,8,4.4,integrale,automatique,13.0036
2,VUS_petit,4,2.0,integrale,automatique,10.1294
3,voiture_compacte,6,3.3,propulsion,automatique,11.4805
4,voiture_deux_places,8,5.0,integrale,automatique,13.0672
5,voiture_minicompacte,3,1.5,traction,automatique,7.49574
6,voiture_minicompacte,3,1.5,traction,manuelle,7.58742
7,VUS_standard,6,3.8,integrale,automatique,11.2005
8,monospace,6,3.3,traction,automatique,11.7801
9,VUS_standard,8,5.0,4x4,automatique,14.7006


In [205]:
function main()
    # println(find_duplicates(trainData))
    uniqueD = get_unique_data(trainData)
    # find_pairs(uniqueD, testData)

    # print(uniqueD)
    # print(trainData[!, Not(:id, :annee)])

    # println(uniqueD)
    predict_with_dups(uniqueD, testData)
    # format_file(predict_with_dups(uniqueD, testData))
    # uniqueD[!,:volume_gaz] = uniqueD[!,:nombre_cylindres] .* uniqueD[!,:cylindree]
    # uniqueD
    
    # display(plot(x=uniqueD.cylindree, y=uniqueD.consommation, color=uniqueD.type))
    # scatter(uniqueD.volume_gaz, uniqueD.consommation, zcolor=uniqueD.type)
    # trainDataStandard = getStandardEncodedData(trainData)

    # predictDataStandard = getStandardEncodedData(testData)
    # evaluate_rmse(trainDataStandard)
    # forest_predict(trainDataStandard,predictDataStandard)
end
main()

LoadError: MethodError: no method matching isless(::String31, ::Float64)

[0mClosest candidates are:
[0m  isless([91m::Missing[39m, ::Any)
[0m[90m   @[39m [90mBase[39m [90m[4mmissing.jl:87[24m[39m
[0m  isless(::Any, [91m::Missing[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mmissing.jl:88[24m[39m
[0m  isless([91m::T[39m, ::T) where T<:Union{Float16, Float32, Float64}
[0m[90m   @[39m [90mBase[39m [90m[4mfloat.jl:548[24m[39m
[0m  ...


In [ ]:
# for type in unique(trainData.type)
#     println(type)
#     data_type = trainData[trainData.type .== type, :]
#     println(combine(groupby(data_type, :transmission), :consommation => mean, :volume_gaz => mean, nrow => :nrow))
#     println()
# end

In [ ]:
# for type in unique(trainData.type)
#     println(type)
#     data_type = trainData[trainData.type .== type, :]
#     display(plot(x=data_type.volume_gaz, y=data_type.consommation))
#     println()
# end

## Exploration des données